# Deep Image Stack Annotation

![](assets/20240723_deep_image_stack_annotation.png)


## Overview

This workflow is an exension of the primary [deep image stack workflow](./workflow_deep-image-stack.ipynb) that adds annotation capabilities.

### Key Software
Alongside [HoloViz](https://github.com/holoviz), [Bokeh](https://holoviz.org/), and several other foundational packages, we make extensive use of key open source libraries to implement our solution, such as:

- **[HoloNote](https://github.com/pydata/xarray):** Provides for annotation capabilities on HoloViews elements.

## Prerequisites and Resources

| Topic | Type | Notes |
| --- | --- | --- |
| [Deep Image Stack Workflow](./workflow_deep-image-stack.ipynb) | Prerequisite | Primary workflow |

## Imports and Configuration

In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import holoviews as hv
import panel as pn

pn.extension()
hv.extension('bokeh')

## Loading and Inspecting the Data

In [ ]:
DATA_PATH = "data/real_miniscope_uint8.zarr"

ds = xr.open_dataset(
    DATA_PATH,
    engine = 'zarr',
    chunks = {'frame': 400, 'height':-1, 'width':-1},  # chunk by sets of complete frames
)
da = ds['varr_ref']
da

## Data Visualization

Let's start with our simple HoloViews based app that is complementary to the other approches in the primary workflow. We will want to customize the positioning of the Here, for a straightforward way get a handle on the slider widget that controls the active frame, we'll use a DynamicMap that calls plot_image every time the slider value change.

In [ ]:
# Link function to create image from player widget frame value
def plot_image(value):
    return hv.Image(da.sel(frame=value), kdims=["width", "height"]).opts(
        title=f'frame = {value}',
        frame_height=da.sizes['height'],
        frame_width=da.sizes['width'],
        cmap = "Viridis",
        clim=(0,20),
        colorbar = True,
        tools=['hover', 'crosshair'],
        toolbar='right',
        scalebar=True,
        scalebar_unit=("µm", "m"), # each data bin is about 1 µm
        apply_hard_bounds=True,
        scalebar_opts={
        'background_fill_alpha': 0.5,
        'border_line_color': None,
        'bar_length': 0.10,
        }
    )

video_player = pn.widgets.Player(
    length =len(da.coords["frame"]),
    interval = 100,  # ms
    value = 250, # start frame
    height=90,
    loop_policy="loop",
    align='center',
)

main_view = hv.DynamicMap(pn.bind(plot_image, video_player.param.value_throttled))

# Frame Number Indicator
frame_reactive = pn.bind(lambda value: f'### frame: {value}', video_player.param.value)
frame_markdown = pn.pane.Markdown(object=frame_reactive)

# Max Over Time Overlay
max_proj_time = da.max('frame').compute().astype(np.float32)
img_max_proj_time = hv.Image(
    max_proj_time, ['width', 'height'], label='Max Over Time').opts(
    frame_height=da.sizes['height'],
    frame_width=da.sizes['width'],
    cmap='magma',
)
alpha_slider = pn.widgets.FloatSlider(start=0, end=1, step=.1, value=0.3, name='Alpha of Max Over Time', align='center')
alpha_slider.jslink(img_max_proj_time, value='glyph.global_alpha')

player_layout = pn.WidgetBox('## Playback', video_player, frame_markdown, horizontal=True, align='start')
alpha_slider_layout = pn.WidgetBox('## Max Overlay', alpha_slider, horizontal=True, align='start')

### Spatial Ranges Annotation Extension

We will now construct an annotator object that enables direct annotation of regions of spatial regions interest within the visual interface utilizing the HoloNote package.

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTable

annotator = Annotator({"height": float, "width": float}, fields=["type"], groupby="type")
color_dim = hv.dim("type").categorize(
    categories={"A": "red", "B": "orange", "C": "cyan"}, default="grey"
)
annotator.style.color = color_dim
annotator.style.alpha=.5
panel_widgets = PanelWidgets(annotator)
table_widget = AnnotatorTable(annotator)
annotator_widgets = pn.WidgetBox('## Annotator', panel_widgets, table_widget, horizontal=True, align='start', scroll=True)

#### Link Annotations to Timeseries

Finally, we will display the mean timeseries of all the data in the annotated spatial regions.

In [ ]:
da.coords["frame"].values[0]

In [ ]:
# Initialize the empty time series plot
curve_opts = dict(
    responsive=True,
    frame_height=da.sizes['height'],
    min_width=300,
    show_legend=False,
    xlabel='frame',
    tools=['hover'],
    line_alpha=.6,
    framewise=True,
    axiswise=True,
)

def plot_ts(event):
    curves = {}
    df = annotator.df
    for i,row in df.iterrows():
        h1, h2, w1, w2 = row[["start[height]", "end[height]", "start[width]", "end[width]"]]
        ds_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        curve = hv.Curve(ds_sel.mean(["height", "width"]), group=row['type'], label=f'{i[:6]}')
        curve = curve.opts(subcoordinate_y=True, color=panel_widgets.colormap[row['type']], **curve_opts)
        curves[f"{row['type']} {i[:6]}"] = curve
    time_series.object = (vline * hv.Overlay(curves, kdims=['curve'])).opts(title='Annotated Mean Timeseries', xlim = (frames[0], frames[-1]))

annotator.on_event(plot_ts)

def plot_frame_indicator_line(value):
    if value:
        return hv.VSpans((value, value)).opts(axiswise=True, framewise=True)

line_opts = dict(color="grey", line_width=5, fill_alpha=.5)
vline = hv.DynamicMap(pn.bind(plot_frame_indicator_line, video_player)).opts(**line_opts)

frames = da.coords["frame"].values
time_series = pn.pane.HoloViews(vline * hv.Curve([]).opts(xlim = (frames[0], frames[-1]),
    title='Create an annotation in the image', **curve_opts))

# WIP
# def highlight_selected(indices):
#     print(indices)
#     if indices:
#         # reset all to original alpha
#         for c in time_series.object:
#             c = c.opts(line_alpha=.6)
        
#         # find and change alpha of selected curve
#         i = indices[0]
#         row = annotator.df.loc[i]
#         time_series.object[f"{row['type']} {i[:6]}"] = time_series.object[f"{row['type']} {i[:6]}"].opts(line_color='black')

# pn.bind(highlight_selected, annotator.param.selected_indices, watch=True) # not triggering in notebook?

main_view_overlay = main_view * img_max_proj_time * annotator
widgets = pn.WidgetBox(annotator_widgets, alpha_slider_layout, player_layout, align='center')

img_stack_app_annot = pn.Column(
    widgets,
    pn.Row(main_view_overlay, time_series)
)

img_stack_app_annot

### Future work: 
- add annotation selection highlighting to the timeseries
- freehanddraw polygon to box annotation, with polygon saved as a displayable field

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTable

annotator = Annotator({"time": float}, fields=["type"], groupby="type")
color_dim = hv.dim("type").categorize(
    categories={"A": "red", "B": "orange", "C": "cyan"}, default="grey"
)
# annotator.style.color = color_dim
# annotator.style.alpha=.5
panel_widgets = PanelWidgets(annotator)
# table_widget = AnnotatorTable(annotator)
# annotator_widgets = pn.WidgetBox('## Annotator', panel_widgets, table_widget, horizontal=True, align='start', scroll=True)

curves = {}
for i in range(4):
    curves[f'{i}'] = hv.Curve(np.random.random(10), 'time', 'val', label=f'{i}').opts(subcoordinate_y=True, alpha=.5)

o = hv.Overlay(curves, kdims=['time', 'curve'])
ts = pn.pane.HoloViews(annotator * o)

def highlight_selected(indices):
    print(annotator.selected_indices)
    ts.object.data = hv.Curve([])
    print(indices)
    ts.object.data
    # if indices:
    #     print(indices[0])
        
    #     # reset all to original alpha
    #     time_series.object.data.opts(line_alpha=.6)
        
    #     # find and change alpha of selected curve
    #     row = annotator.df.loc[indices[0]]
    #     time_series.object.data[f"{row['type']} {i[:6]}"] = time_series.object.data[f"{row['type']} {i[:6]}"].opts(line_alpha=1)

annotator.param.selected_indices.rx.watch(highlight_selected)

# a = pn.bind(highlight_selected, , watch=True)

pn.Column(panel_widgets, ts).show()

In [ ]:
# from holonote.annotate import Annotator
# from holonote.app import PanelWidgets
# import xarray as xr
# import panel as pn
# import holoviews as hv

# hv.extension("bokeh")

# ds = xr.tutorial.open_dataset("air_temperature")


# def plot_image(time):
#     image = hv.Image(ds.sel(time=time), ["lon", "lat"], ["air"]).opts(
#         cmap="RdBu_r", title=time, colorbar=True, clim=(220, 340), width=500, height=300
#     )
#     return image


# def plot_timeseries_by_select(indices):
#     if indices:
#         row = annotator.df.loc[indices[0]]
#         lon1 = row["start[lon]"]
#         lon2 = row["end[lon]"]
#         lat1 = row["start[lat]"]
#         lat2 = row["end[lat]"]
#         ds_sel = ds.sel(lon=slice(lon1, lon2), lat=slice(lat2, lat1)).mean(
#             ["lat", "lon"]
#         )
#         time_series.object = hv.Curve(ds_sel["air"]).opts(
#             title=f"Time Series {lon1:.2f} {lat1:.2f} {lon2:.2f} {lat2:.2f}", width=500
#         )


# def plot_timeseries_by_stream(bounds):
#     if not bounds:
#         lon1, lat1, lon2, lat2 = ds.lon.min(), ds.lat.min(), ds.lon.max(), ds.lat.max()
#         ds_sel = ds
#     else:
#         lon1, lat1, lon2, lat2 = bounds
#         ds_sel = ds.sel(lon=slice(lon1, lon2), lat=slice(lat2, lat1))
#     time_series.object = hv.Curve(ds_sel["air"].mean(["lat", "lon"])).opts(
#         title=f"Time Series {lon1:.2f} {lat1:.2f} {lon2:.2f} {lat2:.2f}", width=500
#     )


# times = ds.time.dt.strftime("%Y-%m-%d %H:%M").values.tolist()

# # start annotation
# annotator = Annotator({"lon": float, "lat": float}, fields=["Description", "Time", "Z"])
# annotator_widgets = PanelWidgets(
#     annotator, field_values={"Time": times, "Z": 100}, as_popup=True
# )

# # make image dependent on the selected time
# time_input = annotator_widgets.fields_widgets[1]
# time_input.value = times[0]
# image = hv.DynamicMap(pn.bind(plot_image, time_input))

# time_series = pn.pane.HoloViews()

# # update plot time when a new box is SELECTED
# pn.bind(plot_timeseries_by_select, annotator.param.selected_indices, watch=True)

# # update plot time when a new box is CREATED
# display = annotator.get_display("lat", "lon")
# box_stream = display._edit_streams[0]  # to make public later
# box_stream.source = image
# pn.bind(plot_timeseries_by_stream, box_stream.param.bounds, watch=True)

# # layout
# pn.Row(annotator_widgets, pn.Column(annotator * image, time_series)).show()


In [ ]:
annotator.selected_indices

In [ ]:
curves

In [ ]:
o

In [ ]:
ts.object.data[(0,)] = ts.object.data[(0,)].opts(line_alpha=.5)
ts